<a href="https://colab.research.google.com/github/alessandroantonucci/CredalCAT/blob/main/Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1HRnKVskd9rWaj-63e9mBE8KcDBXEdY6w?usp=sharing

In [70]:
import numpy as np
from math import log
from random import random

def updater(prior,cpts,questions,answers):
  posterior = np.array([prior,1.0-prior])
  for (q,a) in zip(questions,answers):
    if a == 0: # Q=0
      posterior[0] *= (1.0-cpts[q][0]) # P(S=1,Q=0) = P(S=1)*P(Q=0|S=1)
      posterior[1] *= (1.0-cpts[q][1]) # P(S=0,Q=0) = P(S=0)*P(Q=0|S=0)
    else: # Updating given a correct answer
      posterior[0] *= cpts[q][0] # P(S=1,Q=1) = P(S=1)*P(Q=1|S=1)
      posterior[1] *= cpts[q][1] # P(S=0,Q=1) = P(S=0)*P(Q=1|S=1)
  return posterior/sum(posterior) # P(S|Q=q) \prop P(S,q)

def entropy(x): # Entropy of [p,1-p]
    return -(x*log(x,2)+(1-x)*log(1-x,2))

def prob_question(p,c):
  # p(Q=1) = p(Q=1|S=1)P(S=1)+P(Q=1|S=0)P(S=0)
  return c[0]*p+c[1]*(1-p)

def expent_scores(p,c,q,a): # expected entropy
  scores = [0.0 for _ in range(len(c))]
  for i in range(len(c)):
    pS = updater(p,c,q,a) # [P(S=1|q=a),P(S=0|q=a)]
    p0 = updater(pS[0],c,q+[i],a+[0]) #[P(S=1|Q=0,q=a),P(S=0|Q=0,q=a)]
    p1 = updater(pS[0],c,q+[i],a+[1]) #[P(S=1|Q=1,q=a),P(S=0|Q=1,q=a)]
    pq = prob_question(pS[0],c[i]) # P(Q=1)
    scores[i] = pq*entropy(p1[0])+(1-pq)*entropy(p0[0]) #H[S|Q]
  return scores

p = 0.5 #P(S=1)
tables = [[0.,0.] for _ in range(4)]
tables[0] = [0.9, 0.4] #p(Q=1|S=1),P(Q=1|S=0)
tables[1] = [0.8, 0.5] 
tables[2] = [0.9, 0.5] 
tables[3] = [0.55, 0.25]

entropy_star = entropy(0.1)
e = 1.0
qst = [] # array with the integers denoting the templates of the ansers
ans = [] # array with the boolean integers denoting the answers to the questions in qst

t = 0
for k in range(10):
#while e > entropy_star:
  posterior = updater(p,tables,qst,ans) #updating given previous answers
  picked_question = np.argmax(expent_scores(p,tables,qst,ans)) # picking the best question
  past = ""
  for (q,a) in zip(qst,ans):
    past += "Q"+str(q)+"="+str(a)+" "
  print("T=%d \t P(S=1|%s)=%2.4f" %(t,past,posterior[0])) # logging
  qst += [picked_question]
  given_answer = 0  
  if random()<prob_question(posterior[0],tables[picked_question]): #simulating the answer
    given_answer = 1
  ans += [given_answer]
  posterior = updater(p,tables,qst,ans) #updating given previous answers
  e = entropy(posterior[0])
  t += 1

T=0 	 P(S=1|)=0.5000
T=1 	 P(S=1|Q3=1 )=0.6875
T=2 	 P(S=1|Q3=1 Q3=1 )=0.8288
T=3 	 P(S=1|Q3=1 Q3=1 Q1=1 )=0.8856
T=4 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 )=0.9457
T=5 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 Q1=1 )=0.9654
T=6 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 Q1=1 Q3=1 )=0.9840
T=7 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 Q1=1 Q3=1 Q3=1 )=0.9926
T=8 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 Q1=1 Q3=1 Q3=1 Q3=0 )=0.9878
T=9 	 P(S=1|Q3=1 Q3=1 Q1=1 Q0=1 Q1=1 Q3=1 Q3=1 Q3=0 Q3=1 )=0.9944
